In [1]:
import sqlite3
import os, sys
import pandas as pd
from datetime import datetime 
import numpy as np
import pickle
import warnings

#data_dir = '/Users/xlx/Downloads/graph-data'
data_dir = '/home/xlx/d2/MicrosoftAcademicGraph'
conf_list = ['MM', 'CVPR', 'NIPS', 'ICML', 'IJCAI', 'PLDI']

paper_db = os.path.join(data_dir, 'Paper.db')

conn = sqlite3.connect(paper_db)
cur = conn.cursor()

load_ref = lambda fn: pd.read_table(fn, header=None, names=['PaperID', 'RefID'])

conf_file = os.path.join(data_dir, 'data_txt', 'Conferences.txt')
conf_df = pd.read_table(conf_file, header=None, names=['ConfID', 'Abbrv', 'FullName'])


In [16]:
## create a new table with 6 columns
"""
paper (id, conf/jnl, year)  ref (id, conf/jnl, year)
"""
# for each conference, 
#for c in conf_list[:1]:
c = conf_list[-1]
row = conf_df.loc[conf_df['Abbrv'] == c]
conf_id = list(row['ConfID'])[0]

output_dir = os.path.join(data_dir, 'out', c)

conf_paper_file = os.path.join(output_dir, 'papers.'+ c +'.txt')
df_paper = pd.read_table(conf_paper_file, header=None, 
                         names=['PaperID', 'TitleOrig', 'TitleNorm', 'PubYear', 'PubDate', 
                               'DOI', 'VenueOrig', 'VenueNorm', 'JournalID', 'ConfID', 'PaperRank' ])
df_paper.head()
set_paper = set(df_paper['PaperID'])

citing_file = os.path.join(output_dir, 'citing.'+c+'.txt')
df_citing = load_ref(citing_file)
cited_file = os.path.join(output_dir, 'cited.'+c+'.txt')
df_cited = load_ref(cited_file)
print ("{} conference {}: {} papers ({}-{})".format(datetime.now(), c, df_paper['PaperID'].count(), 
                                               df_paper['PubYear'].min(), df_paper['PubYear'].max()))
print ("\t citing {} papers, cited by {}".format(df_citing['PaperID'].count(), df_cited['PaperID'].count()))

# left joins for both the citing and cited
dfx_citing = df_citing.merge(df_paper[['PaperID', 'PubYear', 'ConfID']], on='PaperID', how='left') 
dfx_citing = dfx_citing.rename(columns = {'PubYear':'PaperPubYear', 'ConfID':"PaperConfID"})
dfx_citing['RefPubYear'] = 1000
dfx_citing['RefVenueID'] = 'AAAAaaaa'

dfx_cited = df_cited.merge(df_paper[['PaperID', 'PubYear', 'ConfID']], 
                           left_on="RefID", right_on='PaperID', how='left') 
dfx_cited.drop('PaperID_y', axis=1, inplace=True)
dfx_cited = dfx_cited.rename(columns = {'PubYear':'RefPubYear', 'ConfID':"RefConfID", "PaperID_x":"PaperID"})
dfx_cited['PaperPubYear'] = 1000
dfx_cited['PaperVenueID'] = 'AAAAaaaa'


2016-04-08 11:42:06.606135 conference PLDI: 1171 papers (1979-2015)
	 citing 22040 papers, cited by 49083


In [19]:
dfx_citing.head()

,PaperID,RefID,PaperPubYear,PaperConfID,RefPubYear,RefVenueID
0,750BF6DA,5A925212,1993,4332078,1989,
1,751854B5,02E6B0E4,2013,4332078,2011,4549948F
2,751854B5,779724BD,2013,4332078,2013,074E39F8
3,751854B5,80BB4591,2013,4332078,2009,4332078E
4,75192A13,5916C0AD,1988,4332078,1981,


In [18]:
# go over the citing db
cnt = 0

for idx, row in dfx_citing.iterrows():
    cur.execute('SELECT * FROM paper_pruned WHERE id=?', (row['RefID'], ) )
    s = cur.fetchone() 
    #dfx_citing['RefPubYear'][idx] = s[1]
    #dfx_citing['RefVenueID'][idx] = s[2]
    dfx_citing.loc[idx, 'RefPubYear'] = s[1]
    dfx_citing.loc[idx, 'RefVenueID'] = s[2]
    cnt += 1
    if cnt % 100 == 0 : # 2000000
        print('{} {:6.0f} / {:6.0f} records'.format(
                datetime.now(), cnt, df_citing['PaperID'].count() ) )
    if cnt >= 1000: #1e9:
        break

print('{} {:6.0f} / {:6.0f} records. Done.\n\n'.format(
                datetime.now(), cnt, df_citing['PaperID'].count() ) )     

2016-04-08 11:42:09.658902     50 /  22040 records
2016-04-08 11:42:09.810734    100 /  22040 records
2016-04-08 11:42:09.981142    150 /  22040 records
2016-04-08 11:42:10.166852    200 /  22040 records
2016-04-08 11:42:10.168863    200 /  22040 records. Done.




In [ ]:
# go over the cited db
cnt = 0

for idx, row in dfx_cited.iterrows():
    cur.execute('SELECT * FROM paper_pruned WHERE id=?', (row['PaperID'], ) )
    s = cur.fetchone() 
    dfx_cited.loc[idx,'PaperPubYear'] = s[1]
    dfx_cited.loc[idx,'PaperVenueID'] = s[2]
    cnt += 1
    if cnt % 1000 == 0 : # 2000000
        print('{} {:6.0f} / {:6.0f} records'.format(
                datetime.now(), cnt, dfx_cited['PaperID'].count() ) )
    if cnt >= 1e9: #1e9:
        break

print('{} {:6.0f} / {:6.0f} records. Done.\n\n'.format(
                datetime.now(), cnt, dfx_cited['PaperID'].count() ) )

In [8]:
pickle.dump({"name":c, 'citing':dfx_citing, "cited":dfx_cited, "paper":df_paper}, 
           open(os.path.join(output_dir, 'cite_records.'+c+".pkl"), 'wb') ) 

In [ ]:
ptr = 0 
line_cnt = 0
citing_cnt = [0, 0]
cited_cnt = [0, 0]
while ptr < len(paper_buf):        

    eol = paper_buf.find('\n', ptr)
    row = paper_buf[ptr:eol].split('\t')
    line_cnt += 1
    ptr = eol + 1
    """ paper table columns
        'PaperID', 'TitleOrig', 'TitleNorm', 'PubYear', 'PubDate', 
        'DOI', 'VenueOrig', 'VenueNorm', 'JournalID', 'ConfID', 'PaperRank'
    """

    cur_pid = row[0]
    r_ref = list(np.nonzero(dfx_citing['RefID'] == cur_pid)[0])
    r_paper  = list(np.nonzero(dfx_cited['PaperID'] == cur_pid)[0])
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
    
        if len(r_ref) >0:
            citing_cnt[0] += 1
            for rid in r_ref:
                # for each paper being cited by any paper in Conf
                dfx_citing['RefPubYear'][rid] = row[3]
                citing_cnt[1] += 1
                if row[9]: # conference
                    dfx_citing['RefVenueID'][rid] = row[9]
                elif row[8]: # journal
                    dfx_citing['RefVenueID'][rid] = row[8]

        if len(r_paper) >0:
            cited_cnt[0] += 1
            for rid in r_paper:
                cited_cnt[1] += 1
                # for each paper citing by any paper in Conf
                dfx_cited['PaperPubYear'][rid] = row[3]
                if row[9]: # conference
                    dfx_cited['PaperVenueID'][rid] = row[9]
                elif row[8]:
                    dfx_cited['PaperVenueID'][rid] = row[8]
        

    if line_cnt % 5000 == 0 : # 2000000
        print('{} {:9.0f} lines; citing {:6.0f}, {:6.0f} unique; {:6.0f} cited, {:6.0f} unique'.format(
                datetime.now(), line_cnt, citing_cnt[1], citing_cnt[0], cited_cnt[1], cited_cnt[0]) )
    if line_cnt >= 1e9: #1e9:
        break


pickle.dump({"name":c, 'citing':dfx_citing, "cited":dfx_cited, "paper":df_paper}, 
           os.path.join(output_dir, 'cite_records.'+c+".pkl"))
print('{} {:9.0f} lines; citing {:6.0f}, {:6.0f} unique; {:6.0f} cited, {:6.0f} unique\n\n'.format(
                datetime.now(), line_cnt, citing_cnt[1], citing_cnt[0], cited_cnt[1], cited_cnt[0]) )
